In [4]:
import pandas as pd
import random
from math import radians, cos, sin, sqrt, atan2
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import joblib

# دالة لحساب المسافة الجغرافية بين نقطتين باستخدام معادلة Haversine
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # نصف قطر الأرض بالكيلومتر
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    return R * 2 * atan2(sqrt(a), sqrt(1 - a))  # النتيجة بالكيلومتر

# توليد عينة واحدة من البيانات لمحاكاة حالة طائرتين
def generate_sample():
    def rand_coord(): return random.uniform(20.0, 30.0)         # إحداثيات الطائرة
    def rand_alt(): return random.randint(3000, 12000)          # الارتفاع بالقدم
    def rand_speed(): return random.randint(200, 900)           # السرعة كم/س
    def rand_heading(): return random.randint(0, 360)           # الاتجاه بالدرجات

    # اختيار سيناريو من عدة حالات واقعية
    scenario = random.choice([
        "safe_clear",         # طائرات بعيدة وآمنة
        "safe_altitude_gap",  # قريبات لكن فرق الارتفاع كبير
        "conflict_close",     # قريبات بالمسافة والارتفاع (خطر واضح)
        "conflict_converge",  # مسارات متقاربة بسرعات متشابهة (تصادم محتمل)
        "safe_opposite",      # قريبين لكن باتجاهين مختلفين
    ])

    # توليد خصائص الطائرة الأولى
    lat1 = rand_coord()
    lon1 = rand_coord()
    alt1 = rand_alt()
    speed1 = rand_speed()
    heading1 = rand_heading()

    # توليد خصائص الطائرة الثانية بناءً على السيناريو
    if scenario == "safe_clear":
        lat2 = lat1 + random.uniform(0.05, 0.1)
        lon2 = lon1 + random.uniform(0.05, 0.1)
        alt2 = alt1 + random.randint(1000, 3000)
        label = 0

    elif scenario == "safe_altitude_gap":
        lat2 = lat1 + random.uniform(0.001, 0.005)
        lon2 = lon1 + random.uniform(0.001, 0.005)
        alt2 = alt1 + random.randint(1500, 3000)
        label = 0

    elif scenario == "conflict_close":
        lat2 = lat1 + random.uniform(0.0005, 0.002)
        lon2 = lon1 + random.uniform(0.0005, 0.002)
        alt2 = alt1 + random.randint(-200, 200)
        label = 1

    elif scenario == "conflict_converge":
        lat2 = lat1 + random.uniform(0.002, 0.004)
        lon2 = lon1 + random.uniform(0.002, 0.004)
        alt2 = alt1 + random.randint(-300, 300)
        heading1 = heading2 = random.randint(90, 110)  # نفس الاتجاه
        speed2 = speed1 + random.randint(-20, 20)
        label = 1

    elif scenario == "safe_opposite":
        lat2 = lat1 + random.uniform(0.001, 0.004)
        lon2 = lon1 + random.uniform(0.001, 0.004)
        alt2 = alt1 + random.randint(800, 2000)
        heading2 = (heading1 + 180) % 360  # عكس الاتجاه
        speed2 = rand_speed()
        label = 0

    else:
        lat2 = rand_coord()
        lon2 = rand_coord()
        alt2 = rand_alt()
        speed2 = rand_speed()
        heading2 = rand_heading()
        label = 0

    # تأكد من وجود السرعة والاتجاه للطائرة الثانية في كل الحالات
    speed2 = speed2 if 'speed2' in locals() else rand_speed()
    heading2 = heading2 if 'heading2' in locals() else rand_heading()

    # حساب الميزات المشتقة
    distance = haversine(lat1, lon1, lat2, lon2)
    alt_diff = abs(alt1 - alt2)
    speed_diff = abs(speed1 - speed2)
    heading_diff = abs((heading1 - heading2 + 180) % 360 - 180)  # فرق الاتجاهات بدقة

    return {
        'lat1': lat1, 'lon1': lon1, 'alt1': alt1, 'speed1': speed1, 'heading1': heading1,
        'lat2': lat2, 'lon2': lon2, 'alt2': alt2, 'speed2': speed2, 'heading2': heading2,
        'distance': distance,
        'alt_diff': alt_diff,
        'speed_diff': speed_diff,
        'heading_diff': heading_diff,
        'label': label
    }

# توليد 5000 عينة من البيانات وحفظها
data = [generate_sample() for _ in range(5000)]
df = pd.DataFrame(data)
df.to_csv("conflict_dataset.csv", index=False)
print("✅ تم حفظ بيانات المحاكاة")

# ===== تدريب النموذج =====

# تحميل البيانات
df = pd.read_csv("conflict_dataset.csv")

# فصل الميزات عن الهدف (label)
X = df.drop(columns=['label'])
y = df['label']

# تطبيق معيار القياس Standardization على الميزات
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# تقسيم البيانات إلى تدريب واختبار
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42)

# تدريب نموذج الانحدار اللوجستي مع زيادة عدد التكرارات
model = LogisticRegression(max_iter=2000)
model.fit(X_train, y_train)

# طباعة دقة النموذج
accuracy = model.score(X_test, y_test)
print(f"✅ دقة النموذج (Accuracy): {accuracy:.3f}")

# حفظ النموذج والمقياس لاستخدامهم لاحقًا في الواجهة
joblib.dump(model, "toweriq_model_prob.pkl")
joblib.dump(scaler, "toweriq_scaler_prob.pkl")
print("✅ تم حفظ النموذج والمقياس بنجاح")


✅ تم حفظ بيانات المحاكاة
✅ دقة النموذج (Accuracy): 1.000
✅ تم حفظ النموذج والمقياس بنجاح


In [7]:
import joblib
import numpy as np
from math import radians, sin, cos, sqrt, atan2

# تحميل المودل والمقياس
model = joblib.load("toweriq_model_prob.pkl")
scaler = joblib.load("toweriq_scaler_prob.pkl")

# دالة لحساب المسافة الجغرافية
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    return R * 2 * atan2(sqrt(a), sqrt(1 - a))

# مثال: مدخلات الطائرتين
sample = {
    "lat1": 24.5,
    "lon1": 46.7,
    "alt1": 9000,
    "speed1": 700,
    "heading1": 90,
    "lat2": 24.502,
    "lon2": 46.702,
    "alt2": 9050,
    "speed2": 710,
    "heading2": 95
}

# الميزات المشتقة
distance = haversine(sample["lat1"], sample["lon1"], sample["lat2"], sample["lon2"])
alt_diff = abs(sample["alt1"] - sample["alt2"])
speed_diff = abs(sample["speed1"] - sample["speed2"])
heading_diff = abs((sample["heading1"] - sample["heading2"] + 180) % 360 - 180)

# تشكيل المصفوفة النهائية
X_input = np.array([[sample["lat1"], sample["lon1"], sample["alt1"], sample["speed1"], sample["heading1"],
                     sample["lat2"], sample["lon2"], sample["alt2"], sample["speed2"], sample["heading2"],
                     distance, alt_diff, speed_diff, heading_diff]])

# مقياس البيانات
X_scaled = scaler.transform(X_input)

# توقع احتمال التصادم
probability = model.predict_proba(X_scaled)[0][1]  # [0][1] = احتمال أن يكون "تصادم"

print(f"🟠 احتمال التصادم: {probability:.2%}")


🟠 احتمال التصادم: 99.97%


C:\Users\saifw\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
